# Data exploration

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

C:\Users\mirei\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\mirei\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


### Import data

In [2]:
nodes_hd = pd.read_csv('final data/graph_nodes_v2024-05-27.csv') #04-02
edges_hd = pd.read_csv("final data/graph_edges_v2024-05-27.csv") #04-02
print(len(nodes_hd))
print(len(edges_hd))

17083
1356448


C:\Users\mirei\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
nodes_ad = pd.read_csv('final data/alz_graph_nodes_v2024-05-25.csv')#"ALZHEIMER/monarch_nodes_v2024-05-25.csv")
edges_ad = pd.read_csv('final data/alz_graph_edges_v2024-05-25.csv')#"ALZHEIMER/monarch_edges_v2024-05-25.csv")
print(len(nodes_ad))
len(edges_ad)

17685


1450686

There is more than one kind of Alzheimer's

In [4]:
alzh_ids = nodes_ad[(nodes_ad.preflabel.str.contains('Alzheimer')) & (nodes_ad.semantic_groups=='DISO')]
print(len(alzh_ids))

24


### Cleaning + Distribution relationships and semantic groups

In [4]:
edges_hd['property_label'] = edges_hd['property_label'].fillna('NA')
edges_hd['property_label'] = edges_hd['property_label'].str.replace('biolink:', '')
edges_hd['property_label'] = edges_hd['property_label'].str.replace('_', ' ')
# print("Relationships in the Huntington's graph")
# print(edges_hd['property_label'].value_counts())

In [5]:
edges_ad['property_label'] = edges_ad['property_label'].fillna('NA')
edges_ad['property_label'] = edges_ad['property_label'].str.replace('biolink:', '')
edges_ad['property_label'] = edges_ad['property_label'].str.replace('_', ' ')
# print("Relationships in the Alzheimer's graph")
# print(edges_ad['property_label'].value_counts())

In [6]:
nodes_ad['semantic_groups'] = nodes_ad['semantic_groups'].fillna('NA')
# in case it is in the other graph
na = nodes_ad[nodes_ad['semantic_groups'] == 'NA'].id
for node in na:
    nodes_ad.loc[nodes_ad.id == node, 'semantic_groups'] = list(nodes_hd[nodes_hd.id == node]['semantic_groups'])[0]

nodes_ad['semantic_groups'] = nodes_ad['semantic_groups'].fillna('NA')
   
# print(nodes_ad['semantic_groups'].value_counts())

In [7]:
nodes_hd['semantic_groups'] = nodes_hd['semantic_groups'].fillna('NA')
# in case it is in the other graph
na = nodes_hd[nodes_hd['semantic_groups'] == 'NA'].id
for node in na:
    nodes_hd.loc[nodes_hd.id == node, 'semantic_groups'] = list(nodes_ad[nodes_ad.id == node]['semantic_groups'])[0]
    
# print(nodes_hd['semantic_groups'].value_counts())

## Iron

In [8]:
def iron_nodes(nodes):
    desc = nodes.dropna(subset=['description']) 
    iron = desc[desc.description.str.contains('iron')]

    name = nodes.dropna(subset=['name']) 
    iron2 = name[name.name.str.contains('iron')]
    
    iron_t = pd.merge(iron, iron2, on=['id'], how='outer', indicator=False)
    return iron_t

In [9]:
iron_ad = iron_nodes(nodes_ad)
iron_hd = iron_nodes(nodes_hd)
print(len(set(list(iron_ad.id)).intersection(list(iron_hd.id))) == len(iron_ad), len(iron_ad))

True 141


In [10]:
# EXPLORATION
# get all the edges iron nodes - 1st neigh
ed_ad = edges_ad[(edges_ad.subject_id.isin(list(iron_ad.id))) | (edges_ad.subject_id.isin(list(iron_ad.id)))]
# connections iron - main seeds
ids = ['MONDO:0007739', 'HGNC:182293', 'MONDO:0004975', 'HGNC:620', 'HGNC:2095', 'HGNC:6893','HGNC:933',' HGNC:613'] #seeds
ed_i_ad = ed_ad[(ed_ad.subject_id.isin(ids)) | (ed_ad.object_id.isin(ids))]
# print(len(ed_i_ad))
# which are the most relevant iron nodes
nodes_iron = nodes_ad[nodes_ad.id.isin(list(ed_i_ad.subject_id)) | nodes_ad.id.isin(list(ed_i_ad.object_id))]
nodes_iron = nodes_iron[~nodes_iron.id.isin(ids)]
len(nodes_iron)

28

In [22]:
def get_neighbor_nodes(ids, edges):
    '''
    Given a list of ids of nodes, get their neighbors
    input:
    ids: list with ids of nodes
    edges: dataframe with the edges
    
    output:
    nodes: set of ids of the neighbors
    '''
    ed_neigh = edges[(edges.subject_id.isin(ids)) | (edges.object_id.isin(ids))]
#     print('Number of connections to the neighbors:', len(ed_neigh))
    nodes = set(list(ed_neigh.subject_id)+list(ed_neigh.object_id))
    return nodes

In [24]:
# CANDIDATE PAIRS
# nodes in cluster = iron + 1st neigh
nodes_i_hd = get_neighbor_nodes(list(iron_hd.id), edges_hd)
print('Initial Iron subgraph nodes:', len(nodes_i_hd))

# nodes neighbors of huntington's (interesting)
nodes_neigh_hd = get_neighbor_nodes(['MONDO:0007739', 'HGNC:4851'], edges_hd)
print("Huntington's/HTT neighbor nodes:", len(nodes_neigh_hd))

# nodes_neigh_hd_2 = get_neighbor_nodes(nodes_neigh_hd, edges_hd)
# print("Huntington's/HTT neighbor's neighbor nodes:", len(nodes_neigh_hd_2))


# only nodes that are iron - neighbors of HD
keep = set(list(nodes_neigh_hd)  + list(iron_hd.id)) #+ list(nodes_neigh_hd_2)
print(len(keep))
candidate_nodes = [node for node in nodes_i_hd if node in keep] 
print(len(candidate_nodes))


############################################################

# # remove leaf nodes
# degree_count = Counter(edges_ed_hd['subject_id']) + Counter(edges_ed_hd['object_id'])
# leaf_nodes = [node for node, degree in degree_count.items() if degree == 1]
# leaf_remove = [node for node in leaf_nodes if node not in list(iron_hd.id)]

# nodes_connect = [node for node in nodes_ed_hd if node not in leaf_remove]
# len(nodes_connect)

Initial Iron subgraph nodes: 6864
Huntington's/HTT neighbor nodes: 501
635
484


In [25]:
# Create a set of already connected pairs
connected_pairs = set(zip(edges_hd['subject_id'], edges_hd['object_id'])) # | set(zip(edges_ad['subject_id'], edges_ad['object_id'])

# Get all pairs that are not already connected
pairs_i = set()
list_ids = candidate_nodes
n = len(list_ids)

for i in range(n):
    id_i = list_ids[i]
    for j in range(i + 1, n):
        id_j = list_ids[j]
        if id_i in list(iron_hd.id) or id_j in list(iron_hd.id): #to reduce size, we constraint one of the nodes must be relevant
            if (id_i, id_j) not in connected_pairs and (id_j, id_i) not in connected_pairs: #check the pair isnt already connected
                pairs_i.add((id_i, id_j))

print(len(pairs_i))

56288


In [26]:
pairs_pred_iron = pd.DataFrame(pairs_i, columns=['subject_id', 'object_id'])
# pairs_pred.info()
pairs_pred_iron.to_csv('pairs_pred_iron_neigh.csv', index=False)  

In [27]:
#### analysis on the nodes selected

test_iron = pairs_pred_iron# pd.read_csv('data/test_iron_reduced.csv')
nodes_iron = set(list(test_iron.subject_id) + list(test_iron.object_id))
print(len(nodes_iron))

# merge with nodes so we can use the properties
# nodes_hd = pd.read_csv('final data-kg/graph_nodes_v2024-05-27.csv') #04-02
nodes_iron_hd1 = nodes_hd[nodes_hd.preflabel.isin(nodes_iron)]
nodes_iron_hd2 = nodes_hd[nodes_hd.id.isin(nodes_iron)]
nodes_iron_hd = pd.merge(nodes_iron_hd1, nodes_iron_hd2, how='outer')
print(len(nodes_iron_hd), len(nodes_iron_hd1)+len(nodes_iron_hd2))

nodes_iron_hd['semantic_groups'] = nodes_iron_hd['semantic_groups'].fillna('NA')
print(nodes_iron_hd['semantic_groups'].value_counts())

484
484 484
GENE    434
DISO     39
ANAT     10
PHYS      1
Name: semantic_groups, dtype: int64


## Metrics

In [ ]:
def create_graph(df, source, target):
    G = nx.DiGraph()
    # Add nodes from the 'source' and 'target' columns
    G.add_nodes_from(df[source])
    G.add_nodes_from(df[target])
    # Add edges from the DataFrame
    edges = [(row[source], row[target]) for index, row in df.iterrows()]
    G.add_edges_from(edges)
    return G

In [ ]:
ad_graph = create_graph(edges_ad, 'subject_id', 'object_id')

In [ ]:
hd_graph = create_graph(edges_hd, 'subject_id', 'object_id')

In [ ]:
def degree(graph, nodes, top, plot=False, title="Huntington's"):
    
    df = pd.DataFrame()
#     print('degree absolute')
    deg = graph.degree()
    deg_s = dict(sorted(deg, key=lambda x:x[1], reverse=True)[:top])
    
#     print('degree centrality - fraction')
    dc = nx.degree_centrality(graph)
    dc = dict(sorted(dc.items(), key=lambda x:x[1], reverse=True)[:top])
 
    for node_id in dc:
        preflabel = nodes.loc[nodes['id'] == node_id, 'preflabel'].values[0] if node_id in nodes['id'].values else None        
        df = df.append({'id': node_id,'preflabel': preflabel,'degree centrality': dc[node_id],'degree': deg[node_id] }, ignore_index=True)
        
    # plot
    if plot == True:
        degrees = dict(deg)
        pos_degree_vals = list(filter(lambda val: val > 0, degrees.values())) # filtering nodes outdegree values with outdegree > 0
        uq_pos_degree_vals = sorted(set(pos_degree_vals)) # getting unique and sorted outdegree values
        hist = [pos_degree_vals.count(x) for x in uq_pos_degree_vals] # counting frequency of each outdegree values

        x = np.asarray(uq_pos_degree_vals, dtype = float)
        y = np.asarray(hist, dtype = float)

        logx = np.log10(x)
        logy = np.log10(y)

        plt.figure(figsize=(10,10))
        plt.xlim(min(logx), max(logx))
        plt.xlabel('log10 (Degree)')
        plt.ylabel('log10 (Number of nodes)')
        plt.title("Degree Distribution of"+title)
        degree_dist = plt.plot(logx, logy, 'o')

    return df

In [ ]:
metrics_hd = degree(hd_graph, nodes_hd, 5, plot=False, title="Huntington's")
metrics_hd

In [ ]:
# metrics_ad = pd.DataFrame()
metrics_ad = degree(ad_graph, nodes_ad, 5, plot=False, title="Alzheimer's")
metrics_ad

In [ ]:
# higher Closeness Centrality score indicates more central
def closeness_centrality(graph, nodes):
    df = pd.DataFrame()
    clos = nx.closeness_centrality(graph)
    close = dict(sorted(clos.items(), key=lambda x:x[1], reverse=True)[:5])
    for node_id in close:
        preflabel = nodes.loc[nodes['id'] == node_id, 'preflabel'].values[0] if node_id in nodes['id'].values else None        
        df = df.append({'id': node_id,'preflabel': preflabel,'closeness': close[node_id] }, ignore_index=True)
        
    return df

In [ ]:
closeness = closeness_centrality(ad_graph, nodes_ad)
closeness

In [ ]:
closeness = closeness_centrality(hd_graph, nodes_hd)
closeness

In [ ]:
# higher betweenness Centrality score indicates more central
def betweenness_centrality(graph, nodes, top=5):
    df = pd.DataFrame()
    betw = nx.betweenness_centrality(hd_graph)
    betweenness = dict(sorted(betw.items(), key=lambda x:x[1])[:top])
    for node_id in close:
        preflabel = nodes.loc[nodes['id'] == node_id, 'preflabel'].values[0] if node_id in nodes['id'].values else None        
        df = df.append({'id': node_id,'preflabel': preflabel,'betweenness': betweenness[node_id] }, ignore_index=True)
        
    return df

In [ ]:
betweenness = betweenness_centrality(ad_graph, nodes_ad, 5)
betweenness

In [ ]:
betweenness = betweenness_centrality(hd_graph, nodes_hd, 5)
betweenness

## Finding commonalities

### Common nodes

In [ ]:
total_r = len(list(nodes_ad.id)+ list(nodes_hd.id))
total = len(set(list(nodes_ad.id)+ list(nodes_hd.id)))
common = (total_r - total)
print('Total with repe:', total_r)
print('Total wo repe:', total)
print('In common ', common)
dis_hd = len(nodes_hd.id) - common
dis_ad = len(nodes_ad.id) - common
print('Distinct HD', dis_hd)
print('Distinct AD', dis_ad)

In [ ]:
nodes_f = pd.merge(nodes_ad, nodes_hd, on=['id'], how='inner', indicator=False)
print(len(nodes_f) == common)
# to correctly check the semantic groups
nodes = nodes_hd[nodes_hd.id.isin(nodes_f.id)]
print(nodes['semantic_groups'].value_counts())

### Common edges

In [ ]:
hd = edges_hd.drop_duplicates()
ad = edges_ad.drop_duplicates()
hd['subject_id'] = hd['subject_id'].astype(str)
hd['object_id'] = hd['object_id'].astype(str)
hd['property_id'] = hd['property_id'].astype(str)

ad['subject_id'] = ad['subject_id'].astype(str)
ad['object_id'] = ad['object_id'].astype(str)
ad['property_id'] = ad['property_id'].astype(str)
print(len(hd), len(ad))

common_rows = hd.apply(tuple, 1).isin(ad.apply(tuple, 1))
hd_distinct = hd[~hd.apply(tuple,1).isin(ad.apply(tuple,1))]
ad_distinct = ad[~ad.apply(tuple,1).isin(hd.apply(tuple,1))]
common = len(common_rows[common_rows == True])
print('In common:', common)
print('Only in HD:', len(hd_distinct), common+len(hd_distinct) == len(hd))
print('Only in AD:',len(ad_distinct), common+len(ad_distinct) == len(ad))

In [ ]:
cc = hd[common_rows]
cc['property_label'].value_counts()

In [ ]:
ad_distinct.to_csv('ad_edges.csv', index=False)

### TO predict

In [ ]:
nodes_predict2 = nodes_hd[~nodes_hd.id.isin(nodes_f.id)]
nodes_predict2.semantic_groups.value_counts()

In [ ]:
nodes_predict2[nodes_predict2.preflabel.str.contains('Hunt')]

In [ ]:
# nodes only in HD graph
nodes_predict2 = nodes_hd[~nodes_hd.id.isin(nodes_f.id)]
ids = list(nodes_predict2.id)

# get all pairs that are not already connected
pairs = []
for i in range(len(ids)):
    for j in range(len(ids)):
        if i < j:
            if len(edges_hd[(edges_hd.subject_id == ids[i]) & (edges_hd.object_id == ids[j])]) == 0:
                if len(edges_hd[(edges_hd.subject_id == ids[j]) & (edges_hd.object_id == ids[i])]) == 0:
                    pairs.append((ids[i], ids[j]))

print(len(pairs))

In [ ]:
pairs_pred = pd.DataFrame(pairs, columns=['subject_id', 'object_id'])
# pairs_pred.info()
pairs_pred.to_csv('pairs_pred.csv', index=False)  

### Common phenotypes

In [12]:
# FIND COMMON PHENOTYPES

# phenotypes directly attached to HD
pheno_h = edges_hd[(edges_hd['property_label'] == 'has phenotype') & (edges_hd['subject_id'] == 'MONDO:0007739')]
phenotypes_h = nodes_hd[nodes_hd['id'].isin(list(pheno_h.object_id))]
# phenotypes_h['preflabel'] = phenotypes_h['preflabel'].apply(lambda x: x.lower() if isinstance(x, str) else x)
print(len(phenotypes_h))

# phenotypes directly attached to AD
pheno_a = edges_ad[(edges_ad['property_label'] == 'has phenotype') & (edges_ad['subject_id'].isin(list(alzh_ids.id)))]
phenotypes_a = nodes_ad[nodes_ad['id'].isin(list(pheno_a.object_id))]
# phenotypes_a['preflabel'] = phenotypes_a['preflabel'].apply(lambda x: x.lower() if isinstance(x, str) else x)
print(len(phenotypes_a))

# both have the same phenotypes
phenotypes = pd.merge(phenotypes_a, phenotypes_h, on=['id'], how='inner', indicator=True)
print(list(phenotypes.preflabel_x))

59
47
['Seizure', 'Agitation', 'Gait disturbance', 'Dystonia', 'Babinski sign', 'Memory impairment', 'Myoclonus', 'Dementia', 'Hallucinations', 'Disinhibition', 'Personality changes']


### Common Treatments

In [ ]:
# treats = 'treats or applied or studied to treat'
drugs_ad = edges_ad[(edges_ad.property_id.str.contains('treats')) & (edges_ad.object_id.isin(list(alzh_ids.id)))]
drugs = nodes_ad[nodes_ad.id.isin(list(drugs_ad.subject_id))]
# drugs

In [ ]:
drugs_hd = edges_hd[(edges_hd.property_id.str.contains('treats'))& (edges_hd.object_id == 'MONDO:0007739')]
drugs_hd = nodes_hd[nodes_hd.id.isin(list(drugs_hd.subject_id))]
# drugs_hd

In [ ]:
skip = (set(list(drugs.id)).intersection(list(drugs_hd.id)))
drugs_predict = pd.DataFrame()
for drug in list(drugs.id):
    if drug not in skip:
        drugs_predict = drugs_predict.append({'subject_id': drug, 'object_id': 'MONDO:0007739'}, ignore_index=True)
drugs_predict.to_csv('pairs_pred_drugs.csv', index=False)

### Different neighbors

In [56]:
# first neighbors: AD -> node // AD -> exclusive ad node

hd_conn = edges_hd[(edges_hd.subject_id == 'MONDO:0007739')|(edges_hd.object_id == 'MONDO:0007739')]
nodes_hd_conn = nodes_hd[(nodes_hd.id.isin(list(hd_conn.subject_id))) | (nodes_hd.id.isin(list(hd_conn.object_id)))]

ad_conn = edges_ad[(edges_ad['object_id'].isin(list(alzh_ids.id))) | (edges_ad['subject_id'].isin(list(alzh_ids.id)))]
print('AD-> node:', len(ad_conn))
# get rid of things we have explored already
ad_conn = ad_conn[(ad_conn.property_id != 'biolink:has_phenotype') & (ad_conn.property_id != 'biolink:treats_or_applied_or_studied_to_treat')]
nodes_ad_conn = nodes_ad[(nodes_ad.id.isin(list(ad_conn.subject_id))) | (nodes_ad.id.isin(list(ad_conn.object_id)))]

diff_ad = nodes_ad_conn[~nodes_ad_conn.id.isin(list(alzh_ids.id))]
print('neighbors:', len(diff_ad))


diff2 = diff_ad[diff_ad.id.isin(list(nodes_hd_conn.id))]
print('neighbors connected to HD:', len(diff2))

diff1 = diff_ad[~diff_ad.id.isin(list(nodes_hd_conn.id))]
diff1 = diff1[diff1.id.isin(list(nodes_hd.id))]
print('neighbors not connected to HD, but in the graph:', len(diff1))

diff = diff_ad[~diff_ad.id.isin(list(nodes_hd.id))]
print('neighbors not in HD graph:', len(diff))

print(list(diff.preflabel))

AD-> node: 171
neighbors: 18
neighbors connected to HD: 2
neighbors not connected to HD, but in the graph: 11
neighbors not in HD graph: 5
['TREM2', 'inherited neurodegenerative disorder', 'hereditary dementia', 'tauopathy', 'autosomal dominant disease']


### Common neighbors

In [39]:
# second neighbors connections only in ad: common node --ad edge--> node (in both graphs)

common_conn = list(nodes_hd[nodes_hd.id.isin(list(nodes_ad_conn.id))].id)

sec_neigh_ad = edges_ad[(edges_ad.subject_id.isin(common_conn))|(edges_ad.object_id.isin(common_conn))]
sec_neigh_hd = edges_hd[(edges_hd.subject_id.isin(common_conn))|(edges_hd.object_id.isin(common_conn))]
sec_only_ad = sec_neigh_ad[~sec_neigh_ad.apply(tuple,1).isin(sec_neigh_hd.apply(tuple,1))]
unseen_hd = sec_only_ad[(sec_only_ad.subject_id.isin(list(nodes_hd.id))) & (sec_only_ad.object_id.isin(list(nodes_hd.id)))]

unseen_hd

,subject_id,property_id,object_id,reference_uri,reference_supporting_text,reference_date,property_label,property_description,property_uri
141234,HGNC:620,biolink:interacts_with,HGNC:25527,https://www.ncbi.nlm.nih.gov/pubmed/21244100,This edge comes from the Monarch Knowledge Gra...,NaN,interacts with,NaN,http://purl.obolibrary.org/obo/biolink_interac...
361990,HGNC:19679,biolink:interacts_with,HGNC:620,https://www.ncbi.nlm.nih.gov/pubmed/21832049,This edge comes from the Monarch Knowledge Gra...,NaN,interacts with,NaN,http://purl.obolibrary.org/obo/biolink_interac...
395104,HGNC:3008,biolink:interacts_with,HGNC:620,https://www.ncbi.nlm.nih.gov/pubmed/21832049,This edge comes from the Monarch Knowledge Gra...,NaN,interacts with,NaN,http://purl.obolibrary.org/obo/biolink_interac...
545263,HGNC:779,biolink:interacts_with,HGNC:620,https://www.ncbi.nlm.nih.gov/pubmed/21832049,This edge comes from the Monarch Knowledge Gra...,NaN,interacts with,NaN,http://purl.obolibrary.org/obo/biolink_interac...
1152138,HGNC:28956,biolink:has_phenotype,HP:0000006,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,has phenotype,NaN,http://purl.obolibrary.org/obo/biolink_has_phe...
